<a href="https://colab.research.google.com/github/Bhavin52/Case_Study/blob/main/fuzzy_Matcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
! pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [76]:
import pandas as pd
import numpy as np

# helpful modules
import fuzzywuzzy
from fuzzywuzzy import process
import chardet

In [78]:
# read in all our data
professors = pd.read_csv("/content/fuzzy.csv",encoding='latin-1') 

In [79]:
# set seed for reproducibility
np.random.seed(0)

text pre-processing


In [80]:
professors.head()

,Country
0,Abatxolo Kalea
1,Agirreburu kalea
2,Agustinen Etorbidea
3,Aita Donostia Kalea
4,Alameda de las Llanas


in country column we want to data Consitentecy

In [81]:
# get all the unique values in the 'Country' column
countries = professors['Country'].unique()

In [82]:
# sort them alphabetically and then take a closer look
countries.sort()
countries

array(['Abatxolo Kalea', 'Agirreburu kalea', 'Agustinen Etorbidea', ...,
       'Zubialde kalea', 'Zubiberri Bidea', 'Zugatzarte Etorbidea'],
      dtype=object)

Germany is two time Present and USA is two time Present with space

In [83]:
# convert to lower case
professors['Country'] = professors['Country'].str.lower()
# remove trailing white spaces
professors['Country'] = professors['Country'].str.strip()

fuzzy matching to correct inconsistent data entry

In [84]:
# get all the unique values in the 'Country' column
countries = professors['Country'].unique()

In [85]:
# sort them alphabetically and then take a closer look
countries.sort()
countries

array(['abatxolo kalea', 'agirreburu kalea', 'agustinen etorbidea', ...,
       'zubialde kalea', 'zubiberri bidea', 'zugatzarte etorbidea'],
      dtype=object)

southKorea and South Korea should be same

In [96]:
# get the top 10 closest matches to "south korea"
matches = fuzzywuzzy.process.extract("Aldea Penedo", countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

# take a look at them
matches = matches[0:,2]

TypeError: ignored

first find the matching Ration as we can see there are more >47 with south korea.so below function is finding the matching score

In [88]:
# function to replace rows in the provided column of the provided dataframe
# that match the provided string above the provided ratio with the provided string
def replace_matches_in_column(df, column, string_to_match, min_ratio = 47):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # let us know the function's done
    print("All done!")

so now we do test

In [89]:
# use the function we just wrote to replace close matches to "south korea" with "south korea"
replace_matches_in_column(df=professors, column='Country', string_to_match="south korea")

All done!


now again check south koria column

In [90]:
# get all the unique values in the 'Country' column
countries = professors['Country'].unique()

# sort them alphabetically and then take a closer look
countries.sort()
countries

array(['abatxolo kalea', 'agirreburu kalea', 'agustinen etorbidea', ...,
       'zubialde kalea', 'zubiberri bidea', 'zugatzarte etorbidea'],
      dtype=object)